# Exploring the Cocktail DB

The intent here is to use the relative proportions of ingredients of cocktails to create a recommendation space for drinks. We'll then create an embedding space for suggesting cocktails that are similar or different. A reach goal of this project is to create a navigable embedding space (e.g., Martini - vodka + rye = Manhattan).

# TODO 
1. ~figure out exceptions where units aren't sepearated from metrics by spaces~
2. ~find unique list of metrics~
3. ~identify "stop words" to just ignore when splitting~
4. ~create conversion scheme between all available metrics (to mL)~
5. ~convert dataframe to mL~
6. ~convert mL dataframe to proportions~
7. ~incorporate nonmetric values...somehow~
8. ~take absolute value of units -- sometimes interprets dashes as negative values~
9. ~convert column headers to generic labels (e.g. "absolut kurrant" to "black current liqueur"~
10. create cache for retrieved files

after beta
1. figure out UI for inputing new drinks
2. create exception for unidentified measures (future-proofing)
3. figure out a better way of noting nonmetric units

In [1]:
import numpy as np
import pandas as pd
import string
import requests
import re

In [43]:
"""
constants for metric conversion

instead of listing out stop words, I'm going to list out compatable metrics
and for each value, iterate through and strip out numerics and operators (./)
reminder to account for plurals (e.g. jigger & jiggers)
"""


# taken from wikipedia
primary_conversion_dict = {
                      'cl': 10,
                      'cup' : 236.59,
                      'dash': .92,
                      'dl'  : 100,
                      'drop' : .05,
                      'fifth': 750,
                      'gal': 3785.412,
                      'tablespoon' : 14.787,
                      'tblsp' : 14.787,
                      'tbsp' : 14.787,
                      'jigger' : 44.3602943, #aka a shot
                      'ml' : 1,
                      'ounce' : 29.574,
                      'oz' : 29.574,
                      'pony': 29.57,
                      'pinch' : .31,
                      'pint' : 568.261,
                      'pt' : 568.261,
                      'qt' : 946.35,
                      'quart' : 946.35,
                      'shot'  : 44.3602943,
                      'snit' : 88.72,
                      'splash' : 5.91,
                      'split' : 177.44,
                      'teaspoon' :  4.93,
                      'tsp' :  4.93,
                      'wineglass': 118.29}

secondary_conversion_dict = {'can': 354.882, #surprisingly decent assumption of a 12 oz can standard
                             'cube': 30,
                             'float' : 29.574, #assuming a float to be a pony/ounce
                             'g' : 1, #assuming gram; here as secondary measure to searc
                             'gr': 1, #assuming this is a misnomer abbeviation for gram
                             'gram' : 1,
                             'in' : 16.387, #this assumes cubic inches, which is almost certainly not the intention of the authors
                             'inch': 16.387}

ml_conversion_dict = {**primary_conversion_dict, **secondary_conversion_dict}

metric_words = list(primary_conversion_dict.keys())
odd_metric_words = list(secondary_conversion_dict.keys())


#since we're converting everything to parts, we'll leave "part" as it's own category
part_words = ['part', 'parts']


# there are some words that are not exact metrics but are still units. 
# for now, these will be listed as "ingrdients present" but not incorporated into ratios
nonmetric_words =  ['1', #count value
                    'bottle', # 12oz beer = 354.882 mL, bottle of wine = 750 mL, bottle of soda = 2000 mL
                    'chunks',
                    'count', #filler word where unit is implied'absent
                    'crushed', #a bad metric for ice
                    'fill',
                    'fillwith',
                    'garnish',
                    'glass',
                    'ground',
                    'juice',
                    'package',
                    'piece',
                    'rim',
                    'slice',
                    'sprig',
                    'stick',
                    'top',
                    'twist',
                    'wedge',
                    'whole']

In [44]:
# class DrinkMenu():
# wrap in a class that stores various conversions of data
# as well as metric and nonmetric words


def get_and_preprocess_data():
    """
    Master function for retrieving and processing data. 
    Accesses thecocktaildb api, retrieves data for all cocktails,
    splits data into values and units, converts all values to mL, 
    and then converts values to proportionate amounts.
    
    Still figuring out what to do with nonmetric units

    Parameters
    ----------
    None


    Returns
    -------
    prop_df : n*m pandas.DataFrame
        dataframe of proportional ingredients by cocktail
        n = cocktails
        m = ingredients


    Example
    -------

    An "A1" cocktail has the following row of ingredients
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  1 3/4 shot   1 Shot          1/4 Shot      1/8 Shot   1

    shots are converted to mL and then standardized to be proportions summing to 1
    count values are listed as TRUE
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  77.63        44.36           11.09         5.545      NONE

    """
    
    #TODO add data cache instead of calling API each time
    print('identifying api endpoints...')
    url_list = create_url_list()
    print('pulling data...')
    cocktail_df = get_cdb_data(url_list)
    print('standardizing data...')
    ml_df = standardize_measures_to_metric(cocktail_df, metric='mL')
    print('converting to proportions...')
    prop_df = standardize_measures_to_props(ml_df)
    
    return(prop_df)
    


def create_url_list():
    """
    Generate list of compatable urls on thecocktaildb.com api

    Parameters
    ----------
    None
    
    Returns
    -------
    url_list : list
        list of all possible search urls by first character
        note: includes some urls that do not exist on API    
    """

    url_list = []
    #source url for searching api by first character
    search_url = 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f='

    #string.printable lists all sets of punctuation, digits, ascii_letters and whitespace
    for i in (string.printable):

        url_list.append(search_url+i)

    return url_list


def get_cdb_data(url_list):
    """
    Retrieves dataset as currently listed on thecocktaildb.com

    Parameters
    ----------
    url_list : list(str)
        list of API urls to search for data from create_url_list

    Returns
    -------
    cocktail_df : pandas.DataFrame
        pandas dataframe of units and measuresments with cocktail names as the index
        and ingredients as column headers

    """

    cocktail_df = pd.DataFrame()

    for url in url_list:
        
        try:
            r = requests.get(url)
            
            if r.json()['drinks']:
                
                for cocktail_dict in r.json()['drinks']: 
                    cdf = get_ingredients_from_cocktail(cocktail_dict)
                    cocktail_df = cocktail_df.append(cdf)
        except:
            pass

    clean_cols = sorted(cocktail_df.columns.dropna())

    return cocktail_df[clean_cols].sort_index().drop_duplicates()


def get_ingredients_from_cocktail(cocktail_dict:dict) -> pd.DataFrame:
    """
    Extract measures of ingredients from a dictionary of ingrediences and measures.

    Parameters
    ----------
    cocktail_dict : dict
        dictionary listing attributes of a single cocktail, extracted from thecocktaildb.com api
        see get_cdb_data

    Returns
    -------
    cdf : pd.DataFrame
        pandas dataframe of cocktails extracted from cocktail_dict
        ingredients listed as columns, with cocktails on each row, and measurement values in each cell
    """

    ingredients = [i for i in cocktail_dict.keys() if i.find('strIngredient') > -1]
    ingredients_list = [cocktail_dict[i] for i in ingredients if i]
    ingredients_list = [t.lower() for t in ingredients_list if t]

    measures = [i for i in cocktail_dict.keys() if i.find('strMeasure') > -1]
    measures_list = [cocktail_dict[m] for m in measures if m]
    measures_list = [t.lower() for t in measures_list if t]


    cdf = pd.DataFrame(dict(zip(ingredients_list,measures_list)), index=[cocktail_dict['strDrink']])

    return cdf



def split_value_measure(measure):
    """
    helper function to divide values from units in cocktail measuremets by individual cell
    
    Parameters
    ----------
    row : pandas.Series
        row of values from a pandas.DataFrame
    
    Returns
    -------
    val : numeric
        numeric value of corresponding unit
    unit : string
        string unit of corresponding value
    """

    measure = _check_separate_numeric(measure)
    measure = _remove_punctuation(measure)

    val = sum([eval(i) for i in measure.split() if list(i)[0].isdigit()])

    if not val: #cases were value is implied, e.g. "ice" or "bottle wine"
        val = 1

    unit = np.array([i.lower() for i in measure.split() if list(i)[0].isalpha()]).flatten()
    unit = ' '.join(unit)

    if not unit:
        unit = 'count' #cases where unit is implied, e.g. "Lemon: 1"
        
    #take abs in case of misinterpretation of m-dash as negative
    return(abs(val), unit)


def split_value_measure_byrow(row):
    """
    helper function to divide units from measurements in cocktails by row
    
    Parameters
    ----------
    row : pandas.Series
        row of values from a pandas.DataFrame
    
    Returns
    -------
    values : list(numeric)
        list of numeric values corresponding to amounts of some unit
    units : list(string)
        list of strings corresponding to units of values
    """
    
    cleanrow = row.dropna()
    values = []
    units = []
    
    for i, v in cleanrow.iteritems():
        
        val, unit = split_value_measure(v)
        # convert_unit
        values += [val]
        units += [unit]
    
    return(values, units)


def convert_unit(value, input_unit, conversion='mL'):
    """
    convert values of input_units to conversion units
    only supports conversion to mL at the moment
    
    Parameters
    ----------
    value : numeric
        numeric value corresponding to amount of some unit
    input_unit : string
        string corresponding to unit of value
    conversion : str
        unit to convert input_unit value
        only mL is supported at the moment
    
    Returns
    -------
    c_values : numeric
        values converted to conversion metric
    """

    if input_unit in part_words:
        c_value = value
    #need to figure out what to do with parametic words
    elif input_unit in nonmetric_words:
        c_value = 1
    elif input_unit in ml_conversion_dict.keys():
        c_value = value*ml_conversion_dict[input_unit]
    else:
        c_value=None

    return c_value


def batch_convert_units(values, input_units, conversion='mL'):
    """
    convert multiple value-unit pairs to conversion units
    only supports conversion to mL at the moment
    
    Parameters
    ----------
    values : list(numeric)
        list of numeric values corresponding to amounts of some unit
    input_units : list(string)
        list of strings corresponding to units of values
    conversion : str
        unit to convert input_unit-value pairs
        only mL is supported at the moment
    
    Returns
    -------
    c_values : numeric
        values converted to conversion metric
    """
    
    c_values = []
    for v, u in zip(values, input_units):
        c_values += [convert_unit(value=v, input_unit=u, conversion=conversion)]
        
    return(c_values)

def standardize_measures_to_props(converted_df):
    """
    Convert uniform measures of ingredients (e.g., all ingredients listed in mL) 
    to proportionate amounts per drink
    
    Still figuring out nonmetric units

    Parameters
    ----------
    converted_df : pd.DataFrame
        dataframe of values in a single metric; output of standardize_measures_to_metric


    Returns
    -------
    prop_df : pd.DataFrame
        dataframe of proportional values for each cocktail (row index)


    Example
    -------

    An "A1" cocktail has the following row of ingredients
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  1 3/4 shot   1 Shot          1/4 Shot      1/8 Shot   1

    shots are converted to mL and then standardized to be proportions summing to 1
    count values are listed as TRUE
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  .56          .32             .08           .04        TRUE

    """
    return (converted_df.fillna(0).T / converted_df.fillna(0).sum(axis=1)).T

def standardize_measures_to_metric(df, metric='mL'):
    """
    Convert raw measures of ingredients to milliliters
    
    Still figuring out what to do with nonmetric units

    Parameters
    ----------
    df : pandas.DataFrame
        dataframe of value-metric pairs, exported from get_cdb_data
    metric: str
        metric to convert value-metric pairs in df. 
        currently only supports mL conversions

    Returns
    -------
    converted_df : pandas.DataFrame
        dataframe of input df converted to metric


    Example
    -------

    An "A1" cocktail has the following row of ingredients
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  1 3/4 shot   1 Shot          1/4 Shot      1/8 Shot   1

    shots are converted to mL and then standardized to be proportions summing to 1
    count values are listed as TRUE
        Gin          Grand Marnier   Lemon Juice   Grenadine  Lemon Peel
    A1  77.63        44.36           11.09         5.545      NONE

    """
    converted_df = pd.DataFrame()
    
    for index, row in df.iterrows():
        row = row.dropna()
        # break out values and units
        ingredients = row.index
        values, units = split_value_measure_byrow(row)
        c_units = [find_unit(u) for u in units]
        c_values = batch_convert_units(values, c_units, conversion='mL')
        temp = pd.DataFrame(dict(zip(ingredients, c_values)), index=[index])
        converted_df = converted_df.append(temp)
        
    return(converted_df)
        
        
def find_unit(unit_string, verbose=False):
    """
    find correct unit embedded in larger string
    
    Parameters
    ----------
    unit_string : str
        unstructured string of unit information
        e.g., 'ounces' or 'fill to top'
    verbose : bool
        flag for printing detailed output on string matching

    Returns
    -------
    unit : str
        structured unit from list of acceptable units for converstion
        if no units match the unit_string, will return None
        
    """
    
    item = _remove_spaces(unit_string)
    unit = None
    
    if item in (metric_words + odd_metric_words + part_words):
        unit = item
        
    elif item in (nonmetric_words):
        # separating non-metric words for future editing
        unit = item
        
    else:
        #imperfect matches
        #prioritizing primary metrics
        matches = [metric for metric in (metric_words + part_words) if metric in item]
        if len(matches) == 1:
            unit = matches[0]
        elif any(matches):
            unit = max(matches,key=len)
            if verbose: print(f'no exact match found for [{item}]; using [{unit}]')
        else:
            #secondary metrics
            matches = [metric for metric in (odd_metric_words) if metric in item]
            if any(matches):
                unit = max(matches,key=len)
                if verbose: print(f'no exact match found for [{item}]; using [{unit}]')
            else:
                # if no unit found, search for nonmetrics
                matches = [metric for metric in (nonmetric_words) if metric in item]
                if any(matches):
                    unit = max(matches,key=len)
                    if verbose: print(f'no exact match found for [{item}]; using [{unit}]')

    if (not unit) and verbose: print(f'no units for [{item}]')
        
    return(unit)  


def _check_separate_numeric(string):
    """
    helper function to make sure that numeric values have a space between alphabetical characters
    
    Parameters
    ----------
    string : str
        any string
        
    Returns
    -------
    new_string : str
        string with guaranteed spaces before and after numeric values
    """

    new_string = string
    space_loc = []

    for i, s in enumerate(string):
        if (s.isalpha() and string[i-1].isnumeric()) or (s.isnumeric() and string[i-1].isalpha()):
            space_loc += [i]

    space_loc.reverse()

    for i in space_loc:
        new_string = new_string[:i] + ' ' + new_string[i:]

    return new_string


def _remove_spaces(string):
    """
    verifies that no spaces exist in string
    removes spaces if found
    
    Parameters
    ----------
    string : str
        any string
    
    Returns
    -------
    new_string : str
        string with spaces removed
    """
    return string.replace(" ", "")

def _remove_punctuation(string):
    """
    remove punctuation from strings
    necessary for separating numerics from alphabetic characters
    
    Parameters
    ----------
    string : str
        any string
    
    Returns
    -------
    new_string : str
        string with punctuation replaced with a space
    """
    return re.sub(r"[+,.;@#?!&$-]+\ *", " ", string)

In [45]:
url_list = create_url_list()
cocktail_df = get_cdb_data(url_list)
cocktail_df.head()

,151 proof rum,7-up,absinthe,absolut citron,absolut kurant,absolut peppar,absolut vodka,advocaat,agave syrup,allspice,...,white rum,white wine,wild turkey,wine,worcestershire sauce,wormwood,yellow chartreuse,yoghurt,yukon jack,zima
1-900-FUK-MEUP,NaN,NaN,NaN,NaN,1/2 oz,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110 in the shade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151 Florida Bushwacker,1/2 oz bacardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155 Belmont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24k nightmare,1/2 oz bacardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
prop_df = get_and_preprocess_data()

identifying api endpoints...
pulling data...
standardizing data...
converting to proportions...


In [11]:
prop_df.head()

,absolut kurant,amaretto,chambord raspberry liqueur,cranberry juice,grand marnier,malibu rum,midori melon liqueur,pineapple juice,lager,tequila,...,limeade,white wine,cream of coconut,agave syrup,root beer,zima,peachtree schnapps,gold rum,pernod,ouzo
1-900-FUK-MEUP,0.2,0.1,0.1,0.2,0.1,0.100000,0.1,0.1,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110 in the shade,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.727273,0.272727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151 Florida Bushwacker,0.0,0.0,0.0,0.0,0.0,0.032258,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155 Belmont,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24k nightmare,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# finding specific  units for debugging

In [16]:
uvals = ['1 bottle', '1 small bottle','2 bottles','1 mikey bottle']

In [17]:
for u in uvals:

    my_data = cocktail_df[cocktail_df == u].to_numpy()

    for idrow, row in enumerate(my_data):
        for idcol, col in enumerate(row):
            if not pd.isnull(col):
                print("Value :"+str(col)+" column:"+str(idcol)+" row:"+str(idrow))
                print(f"drink:{cocktail_df.index[idrow]} \t ingredient: {cocktail_df.columns[idcol]}")

Value :1 bottle column:60 row:168
drink:Halloween Punch 	 ingredient: cherry juice


In [47]:
cocktail_df.loc['Rum Punch'].dropna()

fruit punch     355 ml frozen 
ginger ale       large bottle 
ice                   crushed 
orange juice    355 ml frozen 
rum              mikey bottle 
Name: Rum Punch, dtype: object

# Cleaning up column semantics

In [1]:
import numpy as np
import pandas as pd
import string
import requests
import re

In [4]:
# importing local files
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../bartindr/')

import data_grabbing, constants

In [6]:
prop_df = data_grabbing.get_and_preprocess_data()

identifying api endpoints...
pulling data...
standardizing data...
converting to proportions...


In [33]:
column_semantics = {'151 proof rum':'rum',
                    '7-up':'lemon-lime soda',
                    'absinthe':'absinthe',
                    'absolut citron': 'lemon liqueur',
                    'absolut kurant': 'currant liqueur',
                    'absolut peppar': 'spicy liqueur',
                    'absolut vodka': 'vodka',
                    'advocaat': 'advocaat',
                    'agave syrup':'agave syrup',
                    'allspice':'allspice',
                    'amaretto':'amaretto',
                    'amaro montenegro': 'amaro',
                    'angostura bitters':'bitters',
                    'anis':'anise liqueur',
                    'anisette':'anise liqueur',
                    'aperol':'aperol',
                    'apfelkorn':'apple liqueur',
                    'apple': 'apple',
                    'apple brandy':'apple liqueur',
                    'apple juice':'apple juice',
                    'applejack':'apple liqueur',
                    'apricot brandy':'apricot liqueur', 
                    'apricot nectar': 'apricot liqueur',
                    'asafoetida':'fennel',
                    'añejo rum':'rum',
                    'bacardi limon' : 'lemon liqueur',
                    'bailey' :'irish cream',
                    'baileys irish cream':'irish cream',
                    'banana':'banana',
                    'banana liqueur' : 'banana liqueur',
                    'beer':'beer',
                    'benedictine':'benedictine',
                    'bitter lemon':'lemon',
                    'bitters':'bitters',
                    'black pepper':'black pepper',
                    'black sambuca':'anise liqueur',
                    'blackberries':'blackberries',
                    'blackberry brandy':'blackberry liqueur',
                    'blackcurrant cordial':'currant liqueur',
                    'blackstrap rum':'rum',
                    'blended scotch':'scotch',
                    'blended whiskey':'whiskey',
                    'blood orange':'orange',
                    'blue curacao':'curacao',
                    'bourbon' : 'whiskey', #may the gods forgive me
                    'brandy' : 'brandy',
                    'brown sugar':'sugar',
                    'butter':'butter',
                    'butterscotch schnapps' : 'butterscotch liqueur',
                    'cachaca':'cachaca',
                    'campari':'campari',
                    'carbonated water': 'soda',
                    'cardamom':'cardamom',
                    'cayenne pepper':'pepper',
                    'celery salt':'celery salt',
                    'chambord raspberry liqueur':'raspberry liqueur',
                    'champagne':'champagne', #using 'champagne' for all bubbly
                    'cherry':'cherry',
                    'cherry brandy':'cherry liqueur',
                    'cherry heering':'cherry liqueur',
                    'cherry juice':'cherry',
                    'cherry liqueur':'cherry liqueur',
                    'chocolate':'chocolate',
                    'chocolate liqueur':'chocolate liqueur',
                    'chocolate syrup':'chocolate',
                    'cider':'apple juice',
                    'cinnamon':'cinnamon',
                    'cloves':'cloves',
                    'club soda':'soda',
                    'coca-cola':'cola',
                    'cocoa powder':'chocolate',
                    'coconut liqueur':'coconut liqueur',
                    'coconut milk':'coconut milk',
                    'coconut syrup':'simple syrup',
                    'coffee':'coffee',
                    'coffee brandy':'coffee liqueur',
                    'coffee liqueur':'coffee liqueur',
                    'cognac':'cognac',
                    'cointreau':'orange liqueur',
                    'condensed milk':'cream',
                    'coriander':'coriander',
                    'corn syrup':'simple syrup',
                    'corona':'beer',
                    'cranberry juice':'cranberry juice',
                    'cranberry vodka':'cranberry liqueur',
                    'cream':'cream',
                    'cream of coconut':'coconut milk',
                    'creme de banane':'banana liqueur',
                    'creme de cacao':'chocolate liqueur',
                    'creme de cassis':'currant liqueur',
                    'creme de mure':'blackberry liqueur',
                    'crown royal':'rum',
                    'cucumber':'cucumber',
                    'cumin seed':'cumin',
                    'daiquiri mix':'simple syrup',
                    'dark creme de cacao':'chocolate liqueur',
                    'dark rum':'rum',
                    'demerara sugar':'sugar',
                    'dr. pepper':'cola',
                    'drambuie':'scotch',
                    'dry vermouth':'vermouth',
                    'dubonnet rouge':'apertif wine',
                    'egg':'egg',
                    'egg white':'egg',
                    'egg yolk':'egg',
                    'elderflower cordial':'elderflower liqueur',
                    'espresso':'coffee',
                    'everclear':'grain spirit',
                    'falernum':'simple syrup',
                    'figs':'figs',
                    'firewater':'grain spirit',
                    'food coloring':'dye',
                    'frangelico':'hazelnut liqueur',
                    'fresh lemon juice':'lemon juice',
                    'fresh lime juice':'lime juice',
                    'fruit':'fruit',
                    'fruit juice':'fruit juice',
                    'fruit punch':'fruit juice',
                    'galliano':'herbal liqueur',
                    'gin':'gin',
                    'ginger':'ginger',
                    'ginger ale':'ginger ale',
                    'ginger beer':'ginger ale',
                    'ginger syrup':'simple syrup',
                    'godiva liqueur':'chocolate liqueur',
                    'gold rum':'rum',
                    'goldschlager':'cinnamon liqueur',
                    'grain alcohol':'grain alcohol',
                    'grand marnier':'orange liqueur',
                    'grape juice':'grape juice',
                    'grape soda':'grape juice',
                    'grapefruit juice':'grapefruit juice',
                    'green chartreuse':'chartreuse',
                    'green creme de menthe':'mint liqueur',
                    'grenadine':'grenadine',
                    'guinness stout':'beer',
                    'half-and-half':'cream',
                    'heavy cream':'cream',
                    'honey':'honey',
                    'honey syrup':'honey',
                    'hot chocolate':'chocolate',
                    'hot damn':'cinnamon liqueur',
                    'hot sauce':'pepper',
                    'ice':'ice',
                    'iced tea':'tea',
                    'irish cream':'irish cream',
                    'irish whiskey':'whiskey',
                    'islay single malt scotch':'scotch',
                    'jack daniels':'whiskey',
                    'jagermeister':'herbal liqueur',
                    'jello':'jello',
                    'jim beam':'whiskey',
                    'johnnie walker':'scotch',
                    'jägermeister':'herbal liqueur',
                    'kahlua':'coffee liqueur',
                    'kiwi':'kiwi',
                    'kiwi liqueur':'kiwi liqueur',
                    'kool-aid':'fruit juice',
                    'lager':'beer',
                    'lavender':'lavender',
                    'lemon':'lemon',
                    'lemon juice':'lemon juice',
                    'lemon peel':'lemon',
                    'lemon-lime soda':'lemon-lime soda',
                    'lemonade':'lemon-lime soda',
                    'light cream':'cream',
                    'light rum':'rum',
                    'lillet':'apertif wine',
                    'lillet blanc':'apertif wine',
                    'lime':'lime',
                    'lime juice':'lime juice',
                    'lime peel':'lime',
                    'lime vodka':'lime liqueur',
                    'limeade':'lime juice',
                    'malibu rum':'coconut liqueur',
                    'mango':'mango',
                    'maple syrup':'simple syrup',
                    'maraschino cherry':'cherry',
                    'maraschino liqueur':'cherry liqueur',
                    'marshmallows':'marshmallows',
                    'melon liqueur':'melon liqueur',
                    'mezcal':'mezcal',
                    'midori melon liqueur':'melon liqueur',
                    'milk':'milk',
                    'mint':'mint',
                    'mountain dew':'lemon-lime soda',
                    'nutmeg':'nutmeg',
                    'olive':'olive',
                    'olive brine':'olive brine',
                    'orange':'orange',
                    'orange bitters':'bitters',
                    'orange curacao':'orange liqueur',
                    'orange juice':'orange juice',
                    'orange peel':'orange',
                    'orange spiral':'orange',
                    'oreo cookie':'cookie',
                    'orgeat syrup':'simple syrup', 
                    'ouzo':'anise liqueur',
                    'papaya':'papaya',
                    'passion fruit juice':'passion fruit juice',
                    'passion fruit syrup':'passion fruit juice',
                    'passoa':'passion fruit liqueur',
                    'peach bitters':'bitters',
                    'peach brandy':'peach liqueur',
                    'peach nectar':'peach liqueur',
                    'peach schnapps':'peach liqueur',
                    'peach vodka':'peach liqueur',
                    'peachtree schnapps':'peach liqueur',
                    'pepper':'pepper',
                    'peppermint extract':'mint',
                    'pepsi cola':'cola',
                    'pernod':'anise liqueur',
                    'peychaud bitters':'bitters',
                    'pina colada mix':'simple syrup',
                    'pineapple':'pineapple',
                    'pineapple juice':'pineapple juice',
                    'pineapple syrup':'simple syrup',
                    'pink lemonade':'lemon-lime soda',
                    'pisang ambon':'banana liqueur',
                    'pisco':'pisco',
                    'pomegranate juice':'grenadine',
                    'port':'apertif wine',
                    'powdered sugar':'sugar',
                    'prosecco':'champagne',
                    'raspberry liqueur':'raspberry liqueur',
                    'raspberry syrup':'simple syrup',
                    'raspberry vodka':'raspberry liqueur',
                    'red chili flakes':'pepper',
                    'red wine':'wine',
                    'ricard':'anise liqueur',
                    'root beer':'cola',
                    'rose':'wine',
                    'rosemary':'rosemary',
                    'rosemary syrup':'simple syrup',
                    'roses sweetened lime juice':'lime juice',
                    'rosso vermouth':'vermouth',
                    'ruby port':'apertif wine',
                    'rum':'rum',
                    'rumple minze':'mint liqueur',
                    'rye whiskey':'whiskey',
                    'salt':'salt',
                    'sambuca':'anise liqueur',
                    'sarsaparilla':'cola',
                    'schweppes russchian':'ginger ale',
                    'scotch':'scotch',
                    'sherry':'apertif wine',
                    'sirup of roses':'simple syrup',
                    'sloe gin':'sloe gin',
                    'soda water':'soda',
                    'sour mix':'simple syrup',
                    'southern comfort':'whiskey',
                    'soy sauce':'soy sauce',
                    'spiced rum':'rum',
                    'sprite':'lemon-lime soda',
                    'st. germain':'elderflower liqueur',
                    'strawberries':'strawberries',
                    'strawberry liqueur':'strawberry liqueur',
                    'strawberry schnapps':'strawberry liqueur',
                    'sugar':'sugar',
                    'sugar syrup':'simple syrup',
                    'surge':'lemon-lime soda',
                    'sweet and sour':'simple syrup',
                    'sweet vermouth':'vermouth',
                    'tabasco sauce':'hot sauce',
                    'tea':'tea',
                    'tennessee whiskey':'whiskey',
                    'tequila':'tequila',
                    'thyme':'thyme',
                    'tia maria':'coffee liqueur',
                    'tomato juice':'tomato juice',
                    'tonic water':'tonic',
                    'triple sec':'orange liqueur',
                    'vanilla':'vanilla',
                    'vanilla extract':'vanilla',
                    'vanilla ice-cream':'ice-cream',
                    'vanilla syrup':'simple syrup',
                    'vanilla vodka':'vanilla liqueur',
                    'vermouth':'vermouth',
                    'vodka':'vodka',
                    'water':'water',
                    'whipped cream':'cream',
                    'whipping cream':'cream',
                    'whiskey':'whiskey',
                    'whisky':'whiskey',
                    'white creme de menthe':'mint liqueur',
                    'white rum':'rum',
                    'white wine':'wine',
                    'wild turkey':'whiskey',
                    'wine':'wine',
                    'worcestershire sauce':'worcestershire sauce',
                    'wormwood':'absinthe',
                    'yellow chartreuse':'chartreuse',
                    'yoghurt':'yogurt',
                    'yukon jack':'whiskey',
                    'zima':'lemon-lime soda'}

def _rename_columns(df, column_semantics):
    """
    helper function to rename the ouput dataframe into common semantics
    e.g., anisette, sambuca and ouzo are all types of anise liqueur,
    """
    simple_df = df.rename(columns=column_semantics)
    simple_df = simple_df.groupby(simple_df.columns, axis=1).sum()
    return(simple_df)

In [19]:
prop_df = prop_df.rename(columns=column_semantics)

In [24]:
simple_df = prop_df.groupby(prop_df.columns, axis=1).sum()

In [25]:
simple_df.sum()

absinthe                 1.332451
advocaat                 0.072999
agave syrup              0.051051
allspice                 0.034729
amaretto                 5.791153
                          ...    
water                   10.413607
whiskey                 21.523536
wine                     3.456920
worcestershire sauce     0.021519
yogurt                   4.139131
Length: 133, dtype: float64

In [31]:
simple_df.sum().sort_values(ascending=False).head(10)

gin               39.176696
rum               38.337531
vodka             25.348766
whiskey           21.523536
orange juice      15.105751
vermouth          11.437557
milk              11.046380
water             10.413607
lemon juice        9.173127
orange liqueur     8.964912
dtype: float64

# testing retrieval from class object

In [49]:
import numpy as np
import pandas as pd
import string
import requests
import re

In [50]:
# importing local files
import sys
sys.path.insert(1, '../bartindr/')
import data_grabbing, constants

In [51]:
import importlib
importlib.reload(data_grabbing)

<module 'data_grabbing' from '../bartindr/data_grabbing.py'>

In [52]:
dataclass = data_grabbing.BarData()

In [53]:
dataclass.gather_data()

identifying api endpoints...
pulling data...
standardizing data...
converting to proportions...
standardizing column names...


In [55]:
dataclass.prop_df.sum()

absinthe                 1.332451
advocaat                 0.072999
agave syrup              0.051051
allspice                 0.034729
amaretto                 5.791153
                          ...    
water                   10.413607
whiskey                 21.523536
wine                     3.456920
worcestershire sauce     0.021519
yogurt                   4.139131
Length: 133, dtype: float64

In [57]:
dataclass.cocktail_df.head()

,151 proof rum,7-up,absinthe,absolut citron,absolut kurant,absolut peppar,absolut vodka,advocaat,agave syrup,allspice,...,white rum,white wine,wild turkey,wine,worcestershire sauce,wormwood,yellow chartreuse,yoghurt,yukon jack,zima
1-900-FUK-MEUP,NaN,NaN,NaN,NaN,1/2 oz,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110 in the shade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151 Florida Bushwacker,1/2 oz bacardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155 Belmont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24k nightmare,1/2 oz bacardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
dataclass.standard_df.head()

,absolut kurant,amaretto,chambord raspberry liqueur,cranberry juice,grand marnier,malibu rum,midori melon liqueur,pineapple juice,lager,tequila,...,limeade,white wine,cream of coconut,agave syrup,root beer,zima,peachtree schnapps,gold rum,pernod,ouzo
1-900-FUK-MEUP,14.787,7.3935,7.3935,14.787,7.3935,7.3935,7.3935,7.3935,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110 in the shade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,473.184,177.444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151 Florida Bushwacker,NaN,NaN,NaN,NaN,NaN,14.7870,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155 Belmont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24k nightmare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
